In financial models, variables are often correlated, and the model must capture this statistical behavior. Variables that are correlated tend to move together but not in a completely predictable manner. Example: Price of two tech stocks are positively correlated. We can assume that the daily price changes in these two stocks are positively correlated.
 - Suppose a client has a portfolio consisting of 12 stocks. Assume unit time increment is one week. Compute the drift and volatility parameters from historical data.
 - Discuss how you would do a simulation-optimization. That is, solve a portfolio optimization problem with correlated stock values.

In [377]:
import pyomo
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from numpy.random import lognormal
import plotly.express as px
import numpy as np
from scipy.optimize import minimize
import simpy
import plotly.graph_objects as go
import plotly.subplots as sp
# Example expected returns (12x1 vector)

##### Loading Historical Data

In [378]:
# import pandas as pd

# Specify the path to the folder that contains the CSV file
folder_path = "/Users/sriharikodam/Documents/IsbAMPBA/Term3/AOS/STOCKS"

# Use the os module to get a list of all CSV files in the folder
import os
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Load the CSV files into a list of DataFrames
dfs = []
symbolLst=[]
for csv in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv))
    symbolLst.append(csv.split(".")[0])
    newdf=df[['Close']]
    dfs.append(newdf)

# Concatenate the DataFrames in the list into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True,axis=1)
final_df.columns=symbolLst

#### Verifying Data

In [379]:
final_df.index=pd.to_datetime(df['Date'].tolist())
final_df.head(10)

,COALINDIA,ADANIENT,ADANIPORTS,ASIANPAINT,APOLLOHOSP,BPCL,RELIANCE,BRITANNIA,BAJFINANCE,CIPLA,BHARTIARTL,BAJAJ-AUTO
2023-02-10,213.050003,1846.949951,553.700012,2790.750000,4358.649902,334.899994,2336.649902,4588.000000,6345.500000,1021.950012,768.599976,3831.649902
2023-02-13,211.750000,1717.650024,565.099976,2777.000000,4268.850098,331.149994,2323.350098,4599.850098,6458.700195,1028.349976,775.200012,3883.500000
2023-02-14,212.899994,1749.699951,569.049988,2786.149902,4487.899902,331.149994,2378.100098,4588.450195,6489.950195,1030.800049,785.400024,3876.100098
2023-02-15,215.449997,1779.099976,577.200012,2805.949951,4643.149902,325.700012,2431.949951,4572.000000,6437.299805,1035.500000,784.450012,3905.399902
2023-02-16,217.500000,1796.599976,578.650024,2833.600098,4614.700195,331.700012,2430.050049,4533.250000,6416.700195,1025.750000,776.799988,3906.600098
2023-02-17,218.300003,1722.699951,579.700012,2825.550049,4610.600098,326.149994,2440.199951,4453.799805,6367.399902,964.000000,779.150024,3894.300049
2023-02-20,214.050003,1621.449951,583.200012,2817.100098,4502.049805,323.250000,2414.399902,4503.350098,6379.250000,965.950012,779.049988,3903.800049
2023-02-21,211.850006,1571.099976,547.099976,2795.949951,4449.100098,320.450012,2434.100098,4484.500000,6198.649902,962.549988,772.150024,3837.100098
2023-02-22,214.899994,1404.849976,551.849976,2705.949951,4448.049805,320.549988,2378.899902,4451.500000,6202.350098,960.299988,762.950012,3840.699951
2023-02-23,217.050003,1382.650024,558.900024,2738.500000,4470.649902,317.549988,2367.500000,4442.549805,6236.250000,961.799988,757.099976,3831.899902


#### Resampling to 5 Working Days

In [380]:
finalDfPerWeek=final_df.resample('5D').mean() #Stock price has been resamples for week
finalDfPerWeek.head(10)

,COALINDIA,ADANIENT,ADANIPORTS,ASIANPAINT,APOLLOHOSP,BPCL,RELIANCE,BRITANNIA,BAJFINANCE,CIPLA,BHARTIARTL,BAJAJ-AUTO
2023-02-10,212.566666,1771.433309,562.616659,2784.633301,4371.799967,332.399994,2346.033366,4592.100098,6431.383463,1027.033346,776.400004,3863.750000
2023-02-15,217.083333,1766.133301,578.516683,2821.700033,4622.816732,327.850006,2434.066650,4519.683268,6407.133301,1008.416667,780.133341,3902.100016
2023-02-20,214.960001,1459.139990,560.609998,2761.479980,4461.919922,320.450000,2395.719971,4454.029981,6242.200000,960.159998,764.830005,3852.689990
2023-02-25,219.000000,1373.883342,605.816671,2834.533284,4405.883301,318.099996,2344.766683,4411.916667,6116.650065,897.483337,744.183329,3677.566650
2023-03-02,224.233332,1823.216675,695.816671,2850.983399,4401.816569,324.016663,2373.383301,4357.150065,6077.250000,878.933349,765.849996,3723.433350
2023-03-07,223.316671,1963.000000,692.083333,2822.649984,4360.133301,324.500000,2366.433350,4303.433431,5869.666504,879.250000,770.816671,3815.533284
2023-03-12,220.862499,1823.850006,673.449982,2832.224976,4323.487427,339.699997,2255.849976,4302.450073,5724.437500,876.875000,757.912506,3765.387451
2023-03-17,216.016668,1835.250000,662.900004,2844.700033,4290.700033,356.666657,2231.350016,4280.333496,5698.366699,864.150004,756.150004,3841.699951
2023-03-22,210.066666,1783.350016,640.683329,2800.016683,4291.500000,346.200002,2242.566732,4219.150065,5691.783203,878.549988,762.583333,3871.599935
2023-03-27,213.912495,1703.687500,622.437515,2773.399963,4310.462524,338.487503,2262.825012,4280.599976,5622.862549,892.287506,750.399994,3840.800049


##### Drift & Volatility in the 12 Stocks price

In [455]:
PctSeries=finalDfPerWeek.pct_change(axis=0)
PctSeriesSamplemean=pd.DataFrame()
for i in range(50):
    SampleDf=PctSeries.sample(50)
    PctSeriesSamplemean=pd.concat([PctSeriesSamplemean,SampleDf.mean()],axis=1) # 12 Stocks mean percentage change which is drift of stocks
    # PctSeriesSamplestd= SampleDf.std() # 12 stock standard deviation of percentage change which is volatility of stocks

# PctSeries=
PctMeanSeries = PctSeriesSamplemean.mean(axis=1)
PctStdSeries= PctSeriesSamplemean.std(axis=1)
DriftData= pd.DataFrame(data= PctMeanSeries,columns=['Drift'])
DriftData['Volatility']= PctStdSeries
DriftData.head(20)
DriftData.to_csv("DriftData.csv")

### Covariance matrix of stock

In [382]:
covariance_matrix=PctSeriesSamplemean.T.cov() #Covariance Matrix of Percentage Change
covariance_matrix

,COALINDIA,ADANIENT,ADANIPORTS,ASIANPAINT,APOLLOHOSP,BPCL,RELIANCE,BRITANNIA,BAJFINANCE,CIPLA,BHARTIARTL,BAJAJ-AUTO
COALINDIA,4.827288e-06,2.836095e-06,1.241455e-06,1.010612e-07,-8.502073e-07,2.376352e-06,1.417120e-06,-1.293456e-07,3.636842e-07,-7.047987e-08,-2.375403e-07,3.986727e-07
ADANIENT,2.836095e-06,2.508850e-05,8.894366e-06,1.357961e-06,-5.358416e-07,6.595422e-07,3.533148e-06,1.740806e-06,3.086553e-06,1.152825e-06,6.693712e-07,2.061791e-06
ADANIPORTS,1.241455e-06,8.894366e-06,8.985465e-06,5.459215e-07,-1.612095e-06,-7.036133e-07,1.707419e-06,-7.939212e-07,5.848660e-07,-1.134810e-06,-1.130876e-06,8.028535e-07
ASIANPAINT,1.010612e-07,1.357961e-06,5.459215e-07,2.522521e-06,3.165641e-07,8.621544e-07,6.644006e-07,9.691800e-07,1.231578e-06,-7.953891e-07,-5.122633e-07,7.151724e-07
APOLLOHOSP,-8.502073e-07,-5.358416e-07,-1.612095e-06,3.165641e-07,2.240069e-06,3.921313e-07,-2.286925e-07,1.116191e-06,3.797077e-07,1.808795e-06,7.042506e-07,-9.999244e-08
BPCL,2.376352e-06,6.595422e-07,-7.036133e-07,8.621544e-07,3.921313e-07,5.552966e-06,7.634715e-07,3.107304e-07,1.007192e-07,6.269405e-07,-2.401105e-08,5.455950e-07
RELIANCE,1.417120e-06,3.533148e-06,1.707419e-06,6.644006e-07,-2.286925e-07,7.634715e-07,3.004647e-06,6.239508e-07,7.868209e-07,5.793672e-07,-2.990409e-07,6.801546e-07
BRITANNIA,-1.293456e-07,1.740806e-06,-7.939212e-07,9.691800e-07,1.116191e-06,3.107304e-07,6.239508e-07,1.718433e-06,1.085569e-06,5.677207e-07,5.063356e-07,5.004996e-07
BAJFINANCE,3.636842e-07,3.086553e-06,5.848660e-07,1.231578e-06,3.797077e-07,1.007192e-07,7.868209e-07,1.085569e-06,3.147824e-06,3.584328e-07,-2.154257e-08,6.899491e-07
CIPLA,-7.047987e-08,1.152825e-06,-1.134810e-06,-7.953891e-07,1.808795e-06,6.269405e-07,5.793672e-07,5.677207e-07,3.584328e-07,5.343406e-06,7.697961e-07,-3.633593e-07


### Correlation Matrix of Stock

In [383]:
PctSeriesSamplemean.T.corr()

,COALINDIA,ADANIENT,ADANIPORTS,ASIANPAINT,APOLLOHOSP,BPCL,RELIANCE,BRITANNIA,BAJFINANCE,CIPLA,BHARTIARTL,BAJAJ-AUTO
COALINDIA,1.000000,0.257710,0.188499,0.028961,-0.258549,0.458983,0.372099,-0.044909,0.093297,-0.013877,-0.088042,0.132507
ADANIENT,0.257710,1.000000,0.592390,0.170700,-0.071477,0.055878,0.406938,0.265123,0.347321,0.099567,0.108827,0.300596
ADANIPORTS,0.188499,0.592390,1.000000,0.114668,-0.359327,-0.099610,0.328604,-0.202042,0.109972,-0.163774,-0.307221,0.195588
ASIANPAINT,0.028961,0.170700,0.114668,1.000000,0.133172,0.230359,0.241333,0.465501,0.437058,-0.216647,-0.262653,0.328828
APOLLOHOSP,-0.258549,-0.071477,-0.359327,0.133172,1.000000,0.111183,-0.088150,0.568907,0.142993,0.522817,0.383180,-0.048788
BPCL,0.458983,0.055878,-0.099610,0.230359,0.111183,1.000000,0.186910,0.100590,0.024090,0.115095,-0.008298,0.169077
RELIANCE,0.372099,0.406938,0.328604,0.241333,-0.088150,0.186910,1.000000,0.274592,0.255843,0.144593,-0.140488,0.286541
BRITANNIA,-0.044909,0.265123,-0.202042,0.465501,0.568907,0.100590,0.274592,1.000000,0.466752,0.187352,0.314542,0.278813
BAJFINANCE,0.093297,0.347321,0.109972,0.437058,0.142993,0.024090,0.255843,0.466752,1.000000,0.087396,-0.009888,0.283980
CIPLA,-0.013877,0.099567,-0.163774,-0.216647,0.522817,0.115095,0.144593,0.187352,0.087396,1.000000,0.271190,-0.114790


#### Objective Function

In [384]:
def portfolio_volatility(weights):
    return -np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))

#### Constraints, Optimization of weights and Simulation of different returns of each stock

In [412]:
expected_returns = np.array(PctMeanSeries.tolist())
MinPortfolioReturns=0.04
PerstockMinAllocation=0.01
PerstockMaxAllocationLst=[i*0.1 for i in range(1,8)]
portfoliomeanRturn,portfoliostdRturn,OptimumWeightsAllocation=[],[],[]
# Example covariance matrix (12x12)
for PerstockMaxAllocation in PerstockMaxAllocationLst:
    covariance_matrix=PctSeriesSamplemean.T.cov().values
    cons_eq = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # Sum of weights equal to 1
    cons_ineq1 = ({'type': 'ineq', 'fun': lambda x: x-PerstockMinAllocation}) # Minimum allocation for one stock is 1%
    cons_ineq2 = ({'type': 'ineq', 'fun': lambda x: PerstockMaxAllocation-x}) # Maximum allocation for one stock is 15%
    cons_ineq3 = ({'type': 'ineq','fun': lambda x: np.dot(expected_returns,x)-MinPortfolioReturns}) # Minimum portfolio returns from portfolio is 50%


    initial_weights = np.ones(12) / 12
    result = minimize(portfolio_volatility, initial_weights, constraints=[cons_eq, cons_ineq1,cons_ineq2,cons_ineq3])
    # The optimized portfolio weights
    optimized_weights = result.x
    # optimized_weights
    Optimum_Weight=pd.DataFrame(data={'Stock':[t for t in list(PctMeanSeries.index)],'Weight':optimized_weights})

    Optimum_Weight['Returns']=expected_returns
    # meanLst=WtmeanLst
    # stdLst=WtStdLst
    returnValLst=[]
    portfolioVal=100000
    # return=np.dot(

    def portfolio(env,returnValLst):
        # returnVal=0
        # stockPriceLst=[]
        while True:
            stockPricertrn=[]
            returnVal=0
            for stockreturnemean,stockreturndeviation in zip(PctMeanSeries,PctStdSeries):
                stockPricertrn.append(np.random.normal(stockreturnemean,stockreturndeviation))
            for wt,stockRtrn in zip(Optimum_Weight['Weight'].tolist(),stockPricertrn):
                # print(wt,stockRtrn)
                returnVal+=wt*portfolioVal*stockRtrn
            # print(returnVal)
            returnPct=((returnVal)/100000)
            returnValLst.append(returnPct)
            yield env.timeout(1)

    env=simpy.Environment()
    np.random.seed(100)
    env.process(portfolio(env,returnValLst))
    env.run(until=100000)

    # returnsData=pd.DataFrame(data=returnValLst,columns=['Returns'])
    # returnsData.describe().T
    portfoliomeanRturn.append(np.mean(returnValLst))
    portfoliostdRturn.append(np.std(returnValLst))
    OptimumWeightsAllocation.append(np.round(optimized_weights,2))
# print(portfoliomeanRturn,portfoliostdRturn)

In [446]:
import plotly.express as px


FinalData=pd.DataFrame(data={'Allocation':PerstockMaxAllocationLst,
                             'MeanRtrn':portfoliomeanRturn,
                             "PortfolioRisk":portfoliostdRturn,
                             "OptimumWeights":OptimumWeightsAllocation})


fig1 = px.line(FinalData, x='Allocation', y='PortfolioRisk', title='Portfolio Diversification vs Risk')
fig2 = px.line(FinalData, x='Allocation', y='MeanRtrn', title='Portfolio Return')
fig = sp.make_subplots(rows=1, cols=2, shared_yaxes=False)
fig.add_trace(fig1.data[0], row=1, col=1)
fig.add_trace(fig2.data[0], row=1, col=2)
fig.update_xaxes(title_text="Allocation", row=1, col=1)
fig.update_xaxes(title_text="Allocation", row=1, col=2)
fig.update_yaxes(title_text="Portfolio Risk", row=1, col=1)
fig.update_yaxes(title_text="Mean Return", row=1, col=2)
fig.update_layout(showlegend=False)  # Hide legend
fig.show()

In [447]:

print(f"The Optimum Allocation of Portfolio where risk is minimized and return is maximized is at max 0.4 for ")
print(f"Risk is {FinalData[FinalData['Allocation']==0.4]['PortfolioRisk'].values[0]*100:.2F} and Return is {FinalData[FinalData['Allocation']==0.4]['MeanRtrn'].values[0]*100:.2F}  ")
# print(len(list(PctMeanSeries.index)),FinalData[FinalData['Allocation']==0.4]['OptimumWeights'].to_list()[0])
pd.DataFrame(data={'Stocks':list(PctMeanSeries.index),'weights':list(FinalData[FinalData['Allocation']==0.4]['OptimumWeights'].to_list()[0])})

The Optimum Allocation of Portfolio where risk is minimized and return is maximized is at max 0.4 for 
Risk is 0.16 and Return is 1.08  


,Stocks,weights
0,COALINDIA,0.40
1,ADANIENT,0.11
2,ADANIPORTS,0.40
3,ASIANPAINT,0.01
4,APOLLOHOSP,0.01
5,BPCL,0.01
6,RELIANCE,0.01
7,BRITANNIA,0.01
8,BAJFINANCE,0.01
9,CIPLA,0.01
